# Overview



In [1]:
import pandas as pd
import re
from glob import glob
import os
from pprint import pprint
from RISparser import readris
import nbib
import numpy as np

In [2]:
pd.options.display.max_columns = None

# Create formatted query

This section takes the journal list and produces a formatted query to search the three major databases.  The procedure involves pasting search operators around the journal names and concatenating everything into a single query that can be copied from the notebook and pasted directly into the search bar.  All searches were performed on September 22, 2021.  

```
df = pd.read_csv('/Users/brian/Coding/COVID-research/data/Journals.csv')
df = df['Journals'].tolist()
```

### _Web of science_
The Web of Science (WoS) was specified to capture all records from January 1, 2014 through September 22, 2021.  The exports were made to include all the cited references to each article.  By doing so, the batch limit for export was limited to 500 records.  The records were downloaded as tab-demilited files. The files were then combined and processed using this notebook.   

```
web_of_science = ' OR '.join(df)
web_of_science = "SO = " + "(" + web_of_science + ")"
```

### _PubMed_ 
PubMed contains only a subset of social work journals.  No date limits were put on the search.  The search produced 10,033 articles, going back to the 1970's.  The maximum number of records that can be extracted is 10,000.  Thus, the search results were sorted in the web interface from newest to oldest, and then the extraction was made.  Thus, 33 articles were not retaiend, but these are the oldest records that will not be used in our current work.  Additonally the current cleaning script also excludes all articles publised before 2014.  The extract from PubMed was in a standard RIS format (\*.nbib).  This file was imported into Zotero, exported as a csv file, and processed with this notebook.  

```
pub_med = pub_med = ["(\"" + k +"\"[Journal])" for k in df]
pub_med = ' OR '.join(pub_med)
```

### _EBSCOhost_

Search was performed on September 22, 2021 using the followign search, capturing everything published since 2014 (inclusive).  For this pull, I extracted article records with all the cited references.  In doing so, the extract batch limit was 500, whereas the article records without the cited references allows for batches of 1,000.  The files were saved in a RIS format, which is used by bibliogrphy managers, like Zotero and EndNote.  To process these files, we used a Python module called `RISparsers`.  This converted the files to a Pandas dataframe, which was then prepared for analysis.   

* Social Work Abstracts
* APA PsycInfo
* Abstracts in Social Gerontology;
* Child Development & Adolescent Studies;
* Family Studies Abstracts;
* Political Science Complete;
* Violence & Abuse Abstracts;
* Women's Studies International

```
ebsco = ["SO " + "\"" + k + "\"" +  " OR" for k in df]
ebsco = " ".join(ebsco) 
ebsco = re.sub("OR$", "", ebsco)
```

# Data Preparation

This section prepares the data for cleaning.  Each 

### Prepare _Web of Science_

In [3]:
# Read and combine the WoS data files
%cd /Users/brian/Coding/Data/SocialWorkJournals/WoS
wos = glob('save*.txt')

wos = pd.DataFrame()

for f in glob("save*.txt"):
    holder = pd.read_table(f, sep = "\t",\
                        engine = "python",\
                        header=0,\
                        usecols=([0,4,7,8,11,12,18,19,20,21,22,23,\
                                 25,26,27,28,29,31,34,40,\
                                 41,42,43,55,60,61,62,65]),\
                        parse_dates = True,\
                        infer_datetime_format = True,\
                        quoting=3,
                        #quotechar='"',
                        #quotechar="\"") 
                        on_bad_lines = "skip")
    wos = wos.append(holder,ignore_index=True)

/Users/brian/Coding/Data/SocialWorkJournals/WoS


In [4]:
# The columns are incorrectly named in the file.  This procedure renames the 
# column retrieved.  This mistake can be observed by looking at the raw
# text files.  
wos.columns = ['authors', 'author_full_name', 'title', 'journal', 'language',
       'document_type', 'keywords_au', 'keywords_journal', 'abstract',
       'author_address', 'corresponding_author', 'email_address', 'ID_ORCID',
       'funding_agency', 'funding_text', 'cited_references',
       'cited_reference_count', 'total_times_cited', 'publisher',
       'journal_abbv', 'journal_abbv_dot', 'publication_date',
       'publication_year', 'early_access_date', 'id_database', 'ID_PubMed',
       'open_access_indicator', 'date_of_data_pull']

# Data identifier
wos['data_source'] = "wos"

# Reduce data to key variables
wos = wos[['authors', 'title', 'journal', 'document_type',\
           'abstract', 'publication_year', 'early_access_date',\
           'data_source', 'publication_date']]

In [5]:
# Extract year digits from string
wos['early_access_date'] = wos['early_access_date'].str.extract('(\d+)').astype(int, errors = 'ignore')
wos['publication_year'] = wos['publication_year'].astype(int, errors = 'ignore')

# Address missing date data by using early access date when publication year is missing
wos['publication_year'] = np.where(wos['publication_year'].isnull(),\
                                   wos['early_access_date'],\
                                   wos['publication_year'])

# Filter to 2014 and later. Convert to integer and then filter
wos['publication_year'] = wos['publication_year'].astype(int)
filter_date = wos['publication_year'] > 2013
wos = wos[filter_date]


filter_doc_types = wos['document_type'].str.contains('article|^review$', case = False) 
wos = wos[filter_doc_types]

In [6]:
wos.drop(columns = ['early_access_date', 'document_type', 'publication_date'], inplace=True)

In [7]:
wos.dtypes

authors             object
title               object
journal             object
abstract            object
publication_year     int64
data_source         object
dtype: object

### Prepare _PubMed_

In [8]:
%cd '/Users/brian/Coding/Data/SocialWorkJournals/PubMed'

refs = nbib.read_file('/Users/brian/Coding/Data/SocialWorkJournals/PubMed/pubmed_2021_09_22.nbib')
pubmed = pd.DataFrame.from_dict(refs)
pubmed.shape

/Users/brian/Coding/Data/SocialWorkJournals/PubMed


(10000, 39)

In [9]:
pubmed.columns

Index(['pubmed_id', 'citation_owner', 'nlm_status', 'last_revision_date',
       'print_issn', 'electronic_issn', 'linking_issn', 'publication_date',
       'title', 'abstract', 'copyright', 'authors', 'language',
       'publication_types', 'electronic_publication_date',
       'journal_abbreviated', 'journal', 'nlm_journal_id', 'pmcid', 'keywords',
       'conflict_of_interest', 'received_time', 'revised_time',
       'accepted_time', 'entrez_time', 'pubmed_time', 'medline_time', 'pii',
       'doi', 'publication_status', 'pages', 'place_of_publication',
       'journal_volume', 'journal_issue', 'grants', 'pmc-release_time',
       'descriptors', 'secondary_source', 'corporate_author'],
      dtype='object')

In [10]:
# Select columns by location
pubmed = pubmed.iloc[ : , [7,8,9,11,13, 16]]

# Rename columns
pubmed.rename(columns = {'publication_types':'document_type'}, inplace=True)

# Create data identifier
pubmed['data_source'] = 'pubmed'

# Filter by date
pubmed = pubmed[pubmed['publication_date'] > "2014-01-01"]

# Create a publication year variable through string extraction
pubmed['publication_year'] = pubmed['publication_date'].str.extract('(\d+)')

In [11]:
# Document types are packed in a list.  The first element indicates 'journal article'
# This procedures unpacks the list into a DataFrame of 4 columns. 
pubmed_docs = pubmed['document_type'].apply(pd.Series)

# Rename the first column of interest
pubmed_docs.rename(columns = {0:'document_type_new'}, inplace=True)

# Drop the other columns
pubmed_docs.drop(columns = [1,2,3], axis = 1, inplace=True)

# Concatenate document type variable with the pubmed data
pubmed = pd.concat([pubmed, pubmed_docs], sort = False)

In [12]:
# Fill in empty document type with "inspect" so it isn't dropped
pubmed['document_type'] = np.where(pubmed['document_type_new'].isnull(),\
                                   "Inspect",\
                                   pubmed['document_type_new'])

# Create a filter of document types for inspection
filter_doc_type = pubmed['document_type'].str.contains('article|study|report|inspect', case=False)

# Apply the filter to the data set
pubmed = pubmed[filter_doc_type]

In [13]:
# Clean up the data set by dropping unnecessary columns
pubmed.drop(columns = ['document_type', 'document_type_new', 'publication_date'], inplace = True)

### Prepare _EBSCOhost_

In [14]:
%cd '/Users/brian/Coding/Data/SocialWorkJournals/EBSCO/'
files = glob('*.ris')

df_list = []
for file in files:
    with open(file, 'r') as bibliography_file:
        entries = readris(bibliography_file)#, skip_missing_tags=True)
	    ## create a dataframe once per loop, only
        df = pd.DataFrame.from_dict(entries)
        ## only use pandas API methods (e.g. `drop`) to operate on entries in bulk
        if 'unknown_tag' in df.columns:
            df = df.drop('unknown_tag', axis=1) # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
        df_list.append(df)

# concatenate data frames once, instead of n times inside a loop
ebsco = pd.concat(df_list) # https://pandas.pydata.org/docs/reference/api/pandas.concat.html


/Users/brian/Coding/Data/SocialWorkJournals/EBSCO


In [15]:
print(*ebsco.columns, sep="\n")

type_of_reference
authors
primary_title
journal_name
alternate_title3
publication_year
volume
number
type_of_work
start_page
end_page
issn
abstract
keywords
notes
url
database_provider
name_of_database
id
year
place_published
publisher
language
accession_number
alternate_title2
author_address
doi
secondary_title
tertiary_title
title
access_date
translated_title


In [16]:
# Select columns of interest by integer location
ebsco = ebsco.iloc[ :,[1,2,3,5,8,12, 19]]

# Rename columns
ebsco.columns = ['authors', 'title', 'journal', 'publication_year',\
                 'document_type', 'abstract', 'year']

# Add data_source identifier
ebsco['data_source'] = 'ebsco'

# # Drop duplicates
# ebsco.drop_duplicates(subset = 'title', inplace=True)

In [17]:
# Extract year variable from a string
ebsco['year'] = ebsco['year'].str[:4]
ebsco['publication_year'] = ebsco['publication_year'].str[:4]

# Use two different date variables to address missing publication year
ebsco['publication_year'] = np.where(ebsco['publication_year'].isnull(), ebsco['year'], ebsco['publication_year'])

In [18]:
# Grab document types of interest and those with missing information. These
# are retained for inspection
ebsco['document_type'] = np.where(ebsco['document_type'].isnull(), "inspect", ebsco['document_type'])
filter_doc_types = ebsco['document_type'].str.contains('article|case study|opinion|inspect', case = False) 
ebsco = ebsco[filter_doc_types]

In [19]:
# Final cleanup of data
ebsco.drop(columns = ['year', 'document_type'], inplace=True)

# Prepare the final database

In [20]:
# Put in order of quality
final = pd.concat([ebsco, wos, pubmed])

### Duplicate and missing values

This procedure is to prepare the data files for text processing.  This initial step involves the following steops:

* Drop duplicate titles and abstracts following the merge of the three data sources;
* Drop entries in which either a title or abstract is missing;
* Strip leading and trailing white space characters
* Create a pre-cleaning data file to allow comparisons of the data after the database has been cleaned.


In [21]:
final.dtypes

authors             object
title               object
journal             object
publication_year    object
abstract            object
data_source         object
dtype: object

In [22]:
# Clean up abstracts by first dropping duplicates and then missing abstracts
final['abstract'] = final['abstract'].astype(str).str.strip()
final.drop_duplicates(subset = ['abstract'], inplace=True)
final.dropna(subset = ['abstract'], inplace = True)

In [23]:
final.shape

(57030, 6)

In [24]:
# Clean up titles by first dropping duplicates and then missing abstracts
final['title'] = final['title'].str.replace("\.$", "", regex=True)
final['title_lower'] = final['title'].str.lower().str.strip().str.replace('[^\w\s]','', regex=True)
final = final.drop_duplicates(subset = ['title_lower'], keep = "first")
final = final.dropna(subset = ['title'])

In [25]:
final['title_lower'].head()

0    practice in a pandemic school social workers a...
1    maternal adverse childhood experience exposure...
2    working with nonoffending caregivers of childr...
3    mental health among black youth experiencing s...
4    a comparison of coping strategies among homele...
Name: title_lower, dtype: object

In [26]:
%cd '/Users/brian/Coding/Data/SocialWorkJournals/'
final.to_csv('journals_pre_clean.csv', index=False)

/Users/brian/Coding/Data/SocialWorkJournals


### Text process journal titles

This section processes the journal titles.  This involves hardcoding the exclusion of journal titles that were captured during our search.  Additionally, as some journals come from different databases, we also observe non-standardized spellings.  The following procedures resolve this problem.  

In [27]:
# Remove journals through hard-coded problems

final['journal'] = final['journal'].str.lower()
final.dropna(subset = ['journal'], inplace=True)

# Standardize journal titles

# Remove leading and trailing white space
final['journal'] = final['journal'].str.strip()

# Standardize titles by converting "and" to "&"
final['journal'] = final['journal'].str.replace("\sand\s", " & ", regex=True)

# Remove parantheses
final['journal'] = final['journal'].str.replace("\(.*\)", "", regex=True).str.strip()

# Remove colon and everything that follows
final['journal'] = final['journal'].str.replace("\:.*", "", regex = True)

# Remove 'the' from title
final['journal'] = final['journal'].str.replace("^the", "", regex=True)


In [28]:
filter_terms = ['profile',\
                'nurses',\
                'social development issues',\
                'social development in africa',\
                'teachers\'s professional development',\
                'visual impairment',\
                'clinical psychology',\
                'humanities',\
                'clinical psychology',\
                'profile',\
                'professional development in education',\
                'teachers',\
                'professional development',\
                'rajagiri',\
                'indigenous social development',\
                'economic & social development',\
                'homeless families',\
                '^social development$',\
                'journal of human services',\
                '^review',\
                'Global Social Welfare',\
                'Journal of Community Practice',\
                'Journal of Social Welfare & Family Law',\
                'Journal of Technology in Human Services',\
                'Journal of Applied Social Science',\
                'Journal of Gay & Lesbian Social Services',\
                'Journal of Health & Human Services Administration',\
                'Scientific Annals of',\
                'Social Welfare Interdisciplinary Approach',\
                'Social Work with People with Learning Difficulties',\
                'Revista',\
                'Social intervention',\
                'Portuguese Journal of Behavioral'
                ]

final = final[~final['journal'].str.contains('|'.join(filter_terms), regex=True, case=False)]
final['journal'] = final['journal'].str.strip()

In [44]:
print(final.journal.unique(), sep="\n")

['families in society' 'journal of social work education'
 'journal of social work practice' 'social work & christianity'
 'child & adolescent social work journal' 'social work education'
 'social work research' 'health & social work' 'social work'
 'children & schools' 'clinical social work journal'
 'european journal of social work' 'social work in public health'
 'journal of baccalaureate social work' 'journal of family social work'
 'journal of human behavior in the social environment'
 'child & family social work' 'british journal of social work'
 'social work in health care' 'journal of teaching in social work'
 'journal of religion & spirituality in social work'
 'human service organizations'
 'journal of social work practice in the addictions'
 'social work in mental health' 'journal of gerontological social work'
 'journal of evidence-based social work' 'affilia'
 'international journal of social welfare'
 'asian social work & policy review' 'social service review'
 'australia

In [30]:



# 'Journal of religion: social thought' has a transposed version, 'Social thought: journal of religion'
final['journal'] = final['journal'].str.replace("social thought", "", regex=True)

# Fix journal title name before cleaning
final['journal'] = final['journal'].str.replace("oncology research & practice", "oncology")

#Remove "& development" from New Zealand journal
final['journal'] = final['journal'].str.replace("zealand social work review", "zealand social work")

final['journal'] = final['journal'].str.replace("^canadian social work$",\
                                                "canadian social work review", regex=True)

final['journal'] = final['journal'].str.replace("management leadership & governance",\
                                                "", regex=True)

final['journal'] = final['journal'].str.replace("hiv-aids",\
                                                "hiv/aids", regex=True)

final['journal'] = final['journal'].str.replace("psychoanalytical",\
                                                "psychoanalytic", regex=True)

final['journal'] = final['journal'].str.replace("work-maatskaplike",\
                                                "work / maatskaplike", regex=True)

final['journal'] = final['journal'].str.replace("global-global",\
                                                "global - global", regex=True)


final['journal'] = final['journal'].str.replace("evidence-informed", "evidence-based", regex=True)

final = final[~final['journal'].str.contains('^social development', regex=True)]

In [31]:
# Additional standardization using generalizable (not hard-coded) solutions. 



# Drop all entries with NA or empty cells
final = final.dropna(subset=['journal'])
final = final[final['journal'] != '']

In [32]:
final.shape

(21592, 7)

### Clean abstracts

Approximately 11k entries have an unspecified `document_type`.  These are limited to the EBSCO data source.  Thus, these missing values on `document_type` are replaced by _Article_, and then inspection is performed to remove editorials, book reviews, and other scientific communications. Note that "case study." is retained.

# To Do

Exclude from abstract:  ```[ABSTRACT FROM AUTHOR]```

In [33]:
# Abstract filter terms
filter_terms = ['an introduction to',\
                'erratum',\
                'correction notice',\
                'editor',\
                'book review',\
                'reviews the book',\
                'review of the book',\
                'obituary',\
                'commentary',\
                'a correction',\
                'reports an error',\
                'reviews the book',\
                'letter'
               ]

#filter_abstracts = final['abstract'].str.lower().str.contains(filter_terms)
final = final[~final['abstract'].str.contains('|'.join(filter_terms),\
                                              regex=True, case=False)]

In [34]:
final.shape

(20206, 7)

In [35]:
# Filter abstracts < 100 characters
final['abstract_length'] = final['abstract'].str.len()
final = final[final['abstract_length'] > 100]

In [36]:
# Exclude book reviews
# Note:  three records exist for an article with "Michael Gove" in the title.  
# These records are temporarily excluded and single record will be added.  

filter_terms = ['editor', 'letter', 'comment', 'michael gove']

final = final[~final['title'].str.contains('|'.join(filter_terms), regex=True, case=False)]

In [37]:
%cd '/Users/brian/Coding/Data/SocialWorkJournals/'

/Users/brian/Coding/Data/SocialWorkJournals


In [38]:
final.to_csv('SocialWork_2014_2021.csv', index=False)